# Scraper for FlavorDB

In [27]:
import requests
import json
import pandas as pd
import csv
import numpy as np

Captures only entities with full data, but it seems incomplete

In [4]:
json_res = requests.get('https://cosylab.iiitd.edu.in/flavordb/entities?entity=%20&category=')
text_res = json.loads(json_res.text)
res = json.loads(text_res)

Store all entities in `data.csv` (prelim, unused data)

In [63]:
data = pd.DataFrame.from_records(res)
data.to_csv('data.csv', index=False)

with open('data.csv') as file:
  csv_reader = csv.DictReader(file)
  data = [row for row in csv_reader]

Read in all available entity information into `data_chemicals.csv` (approx. 35 minutes on M1; USED)

In [5]:
# molecules stored in indices from 0 to 978
id = 0
with open('data_chemicals.csv', 'a') as csvfile:
  writer = csv.writer(csvfile)
  # csv header
  writer.writerow(['id', 'common_name', 'category', 'chemicals'])
  while id <= 978:
    try:
      entity_res1 = requests.get(f'https://cosylab.iiitd.edu.in/flavordb/entities_json?id={id}')
      entity_res = json.loads(entity_res1.text)
      category = entity_res['category']
      common_name = entity_res['entity_alias_readable']
      # !! May want to grab functional groups here from entity_res['molecules'][i]['functional_groups']
      molecules = list(map(lambda obj: obj['common_name'], entity_res['molecules']))
      writer.writerow([id, common_name, category, str(molecules)])
    except:
      print(id)
    finally:
      id += 1

406
407
414
420


Removing odor entities with identical chemical makeups

In [9]:
import ast

data = []
with open('data_chemicals.csv', 'r') as file:
  csv_reader = csv.DictReader(file)
  with open('data_final.csv', 'w') as write_file:
    writer = csv.writer(write_file)
    seen = set()
    for row in csv_reader:
      chemicals = row['chemicals']
      # remove entities with duplicate chemical sets
      if chemicals not in seen:
        seen.add(chemicals)
        writer.writerow(row.values())
        # convert stringified list back into python list
        data.append(ast.literal_eval(chemicals))

# getting distinct chemicals
chemical_set = set()
for lst in data:
  chemical_set.update(lst)

print(len(chemical_set))


1781


One-hot encode `data_one_hot.csv`

In [29]:
df = pd.read_csv('data_final.csv', names=['index', 'Name', 'type', 'chems'], header=None)
df['chems'] = df['chems'].apply(ast.literal_eval)

In [33]:
chemical_set

chemical_match = {chemical : i for i, chemical in enumerate(chemical_set)}

In [34]:
x = np.zeros((500, len(chemical_set)))

for i, lst in enumerate(df['chems']):
  for item in lst:
    x[i][chemical_match[item]] = 1

In [55]:
pd.concat([df, pd.DataFrame(x, columns=chemical_match)], axis=1).to_csv('data_binary.csv')

TODO
- append molecular names to cids from `molecules.csv`
- save as pickle file the binary encodings (500 x 1700)
- 1.1: Create uniform fixed dataset (sum 2 binary encodings) --> pre-train on one odor encodings, pass into multiclass classifier
- 1.2: Create uniform distributed dataset, generate a lot more data/mixture

- 2.1: Preset (gamma?) fixed dataset
- 2.2: Gamma distributed dataset

Save pickled inputs (500 x 1781)

In [56]:
import pickle as pkl
with open('binary_opens.pkl', 'wb') as writefile: 
  pkl.dump(x.astype(int), writefile)